# Crawl data with python


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep
import csv
import os
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine
import urllib

driver = webdriver.Chrome()
url = "https://www.cgv.vn/en/movies/now-showing.html"
driver.get(url)
wait = WebDriverWait(driver, 10)

wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'product-image')))
page_source = BeautifulSoup(driver.page_source)
movies = page_source.find_all('a', class_='product-image')
movie_links = [a['href'] for a in movies]
movie_data = []
for x in movie_links:
    driver.get(x)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'product-name')))
    sleep(2) 
    page_source = BeautifulSoup(driver.page_source,"html.parser")
    
    image_tag = page_source.find_all('div', class_='product-image-gallery')
    image_tmp = image_tag[0].find('img')
    image_link = image_tmp['src']
    
    info_name = page_source.find_all('div', class_='product-name')
    name = info_name[0].find('span').get_text().strip()

    info_director = page_source.find_all('div', class_='movie-director movie-info')
    director = info_director[0].find('div').get_text().strip()

    info_actress = page_source.find_all('div', class_='movie-actress movie-info')
    actress = info_actress[0].find('div').get_text().strip()

    length = info_actress[1].find('div').get_text().strip()

    info_genre = page_source.find_all('div', class_='movie-genre movie-info')
    genre = info_genre[0].find('div').get_text().strip()

    info_release = page_source.find_all('div', class_='movie-release movie-info')
    release_raw = info_release[0].find('div').get_text().strip()

    # Convert release date to dd-MM-yyyy format
    release = datetime.strptime(release_raw, '%b %d, %Y').strftime('%Y-%m-%d')




    info_description = page_source.find_all('div', class_='product-collateral toggle-content tabs tabs-format-cgv')
    description = info_description[0].find('div',class_='std').get_text().strip()

    info_trailer = page_source.find_all('div', class_='std product_view_trailer')
    if(info_trailer == []):
        trailer_link = 'No trailer'
    else:    
        iframe_tag = info_trailer[0].find('iframe')

        trailer_link = iframe_tag['src']
        if trailer_link.startswith('//'):
            trailer_link = 'https:' + trailer_link


    movie_data.append({
            'movie_poster': image_link,
            'movie_name': name,
            'movie_description': description,
            'movie_trailer': trailer_link,
            'movie_actress': actress,
            'movie_genres': genre,
            'movie_release': release,
            'movie_length': length,
            'movie_director': director
        })
    print('Name:', name)
    print('Director:', director)
    print('Actress:', actress)
    print('Length:', length)
    print('Genre:', genre)
    print('Release:', release)
    print('Description:', description)
    print('Trailer:', trailer_link)
    print('Image:', image_link)
    print('-----------------------------------')
    sleep(5)

    # Write to CSV file
    output_dir = os.path.expanduser('~')  # This will save to the user's home directory
    csv_file = os.path.join(output_dir, 'now_showing_movies.csv')
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=movie_data[0].keys())
        writer.writeheader()
        writer.writerows(movie_data)


server = 'localhost'  
database = 'CINEMA_TICKET_MANAGEMENT_SYSTEM'
username = 'sa'
password = '29032004'

# Connection string for SQL Server
connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
connection_url = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}"


engine = create_engine(connection_url)


df = pd.DataFrame(movie_data)

# df['movie_release'] = pd.to_datetime(df['movie_release'], errors='coerce').dt.strftime('%Y-%m-%d')

table_name = 'movies'
df.to_sql(table_name, con=engine, if_exists='append', index=False)




Name: DESPICABLE ME 4
Director: Chris Renaud, Patrick Delage
Actress: Steve Carell, Kristen Wiig, Joey King
Length: 94 minutes
Genre: Adventure, Animation, Comedy
Release: 2024-07-05
Description: Gru is facing a new enemy, Maxime Le Mal and Valentina is plotting revenge against him, forcing his family to flee elsewhere. Alongside the struggle to protect his family, Gru must also find common ground with the newest member of the household, Gru Jr.
Trailer: https://www.youtube.com/embed/S1dnnQsY0QU?rel=0&showinfo=0
Image: https://iguov8nhvyobj.vcdn.cloud/media/catalog/product/cache/3/image/c5f0a1eff4c394a251036189ccddaacd/d/m/dm4_posterminion_700x1000.jpg
-----------------------------------
Name: A QUIET PLACE: DAY ONE
Director: Michael Sarnoski
Actress: Joseph Quinn, Alex Wolff, Djimon Hounsou
Length: 99 minutes 17 seconds
Genre: Horror
Release: 2024-06-28
Description: Experience the day the world went quiet.
Trailer: https://www.youtube.com/embed/9wVj8AxpT8A?rel=0&showinfo=0
Image: http

21